# This requires the full stats files of individual runs which are not included in the drop.

In [1]:
%run ../includes/common-20230414.ipynb

CPython 2024-01-04T04:29:57.944278


CPU times: user 394 µs, sys: 469 µs, total: 863 µs
Wall time: 1.08 ms


In [2]:
dfc_raw = get_df()

postproc failed


In [3]:
dfs_01 = get_data(dfc_raw, default_sample_ratio=.1)

9065


/tmp/ipykernel_47632/3317348687.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  best_pf_option_ = best_pf_option.idxmin(skipna=skipna)
/tmp/ipykernel_47632/3317348687.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  best_pf_option_ = best_pf_option.idxmin(skipna=skipna)


In [4]:
def relabel_leg_part(mapping=None, ax=None, **kwargs):
    ax = ax or plt.gca()
    h, labels = ax.get_legend_handles_labels()
    labels = [mapping[x.split('@')[0]] for x in labels]
    return plt.legend(h, labels, **kwargs)


In [5]:
def shorten_ticks():
    for ax in plt.gcf().get_axes():
        locs = ax.get_xticks()
        order = ax.get_xticklabels()
        ax.set_xticks(locs, [x.get_text().replace("Baleen (", "Baleen\n(") for x in order],
           va='center', ha='right', rotation_mode='anchor', rotation=90)


In [6]:
dtx = filter_df(dfc_raw, {'Region': 'Region2', 'Target Cache Size': ef.DEFAULT_CSIZE,
                          'AdmissionPolicyLabel': ['Baleen', 'RejectX'], 'CanonTrace': True,
                          'ShortLabel': ['RejectX', 'Baleen (No Prefetch)', 'Baleen (ML Prefetch)', 'Baleen (All on Partial Hit)'],
                          'PracticalAP': True, 'CanonExp': True, 'DWPDNotFar': True,
                          'Target DWPD': 7.5, 'SampleRatio': 0.1, 'SampleStart': 0})
new_cols = dtx.apply(processors.maxstats, axis=1, result_type='expand')
dtx_ = pd.concat([dtx, new_cols], axis='columns')
dtx_ = dtx_.drop_duplicates(subset=['ShortLabel'])
# TODO: Do it based on sample etc 
reload(processors)
# constrained_layout
processors.plot_breakdowns(dtx_, x='ShortLabel', subplots_order=['RejectX', 'Baleen (All on Partial Hit)'], #'Baleen (ML Prefetch)'],
                order=None,
                sample_ratio=.1, xperiod=600,
                figsize=(6.4,4.8*1.5),
                # max_y=60,
                leg_kwargs=dict(bbox_to_anchor=[.5, 0], loc='upper center', frameon=True, facecolor='white', framealpha=1),
                stack_order=[processors.stonbw_label, processors.stonpf_label, processors.stonios_label])
shorten_ticks()

# savefig("peak_breakdowns", "Region1_wr-34_st-util")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
dtx = filter_df(dfc_raw, {'Region': 'Region4', 'Target Cache Size': ef.DEFAULT_CSIZE,
                          'AdmissionPolicyLabel': ['Baleen', 'RejectX'], 'CanonTrace': True,
                          'ShortLabel': ['RejectX', 'Baleen (No Prefetch)', 'Baleen (ML Prefetch)', 'Baleen (All on Partial Hit)'],
                          'PracticalAP': True, 'CanonExp': True, 'DWPDNotFar': True,
                          'Target DWPD': 7.5, 'SampleRatio': 0.1, 'SampleStart': 0})
new_cols = dtx.apply(processors.maxstats, axis=1, result_type='expand')
dtx_ = pd.concat([dtx, new_cols], axis='columns')
dtx_ = dtx_.drop_duplicates(subset=['ShortLabel'])
# TODO: Do it based on sample etc 
reload(processors)
# constrained_layout
processors.plot_breakdowns(dtx_, x='ShortLabel', subplots_order=['RejectX', 'Baleen (All on Partial Hit)'], #'Baleen (ML Prefetch)'],
                order=None,
                sample_ratio=.1, xperiod=600,
                figsize=(6.4,4.8*1.5),
                # max_y=60,
                leg_kwargs=dict(bbox_to_anchor=[.5, 0], loc='upper center', frameon=True, facecolor='white', framealpha=1),
                stack_order=[processors.stonbw_label, processors.stonpf_label, processors.stonios_label])
shorten_ticks()